## Two stage flair training

In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-zki_cug5
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-zki_cug5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 8.8MB/s 
     |████████████████████████████████| 798kB 52.2MB/s 
     |████████████████████████████████| 256kB 55.9MB/s 
     |████████████████████████████████| 665kB 56.7MB/s 
     |████████████████████████████████| 1.1MB 50.4MB/s 
     |████████████████████████████████| 890kB 62.3MB/s 
     |████████████████████████████████| 3.8MB 41.2MB/s 
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=148939 sha256=d559f0ee990a2f78191d4ada138f404d9c8b3ae627441aefdfd517917df0004c
  Stored in directory: /tmp/pip-ephem-wheel-cache-4g9ryz0s/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Create

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# data_folder = "./drive/My Drive/Capstone/undersampled/housing"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv("/content/drive/My Drive/Capstone/combined_benchmark.csv")

In [0]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
data_folder = "/content/drive/My Drive/Capstone/"

In [0]:
benchmark = benchmark.sample(frac=1, random_state = 42)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

#### Build corpus

In [0]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-28 03:09:17,382 Reading data from /content/drive/My Drive/Capstone
2020-05-28 03:09:17,384 Train: /content/drive/My Drive/Capstone/train.csv
2020-05-28 03:09:17,385 Dev: /content/drive/My Drive/Capstone/dev.csv
2020-05-28 03:09:17,392 Test: /content/drive/My Drive/Capstone/test.csv


#### Create word embeddings

In [6]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-28 05:10:16,328 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpuscu1esm


100%|██████████| 19689779/19689779 [00:00<00:00, 72288885.19B/s]

2020-05-28 05:10:16,739 copying /tmp/tmpuscu1esm to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2020-05-28 05:10:16,760 removing temp file /tmp/tmpuscu1esm


2020-05-28 05:10:34,225 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpuvj88orn


100%|██████████| 19689779/19689779 [00:00<00:00, 66757027.80B/s]

2020-05-28 05:10:34,619 copying /tmp/tmpuvj88orn to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2020-05-28 05:10:34,643 removing temp file /tmp/tmpuvj88orn


#### First Stage Fine-tuning

In [7]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
# classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
# trainer = ModelTrainer(classifier, corpus)
# trainer.train(data_folder, max_epochs=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


#### Data folder after first stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.34.08%20PM.png?token=AAAAOMWRKXQXADFFGZWV72S62WILA) 

### Second Stage GDP (fine tune on hand annotated datasets)

#### Build corpus

In [0]:
new_data_folder = '/content/drive/My Drive/Capstone/undersampled/interest_rates/'


In [0]:
# train_load = pd.read_csv('/content/drive/My Drive/Capstone/undersampled/housing/train.csv')
# train_df= train_load[['title_desc_sent_1', 'title_desc']]
# train_df.columns= ['label', 'text']
# train_df.tail()
# dev_load = pd.read_csv('/content/drive/My Drive/Capstone/undersampled/housing/dev.csv')
# dev_df = dev_load[['title_desc_sent_1', 'title_desc']]
# dev_df.columns= ['label', 'text']
# dev_df.tail()
# test_load = pd.read_csv('/content/drive/My Drive/Capstone/undersampled/housing/test.csv')
# test_df = test_load[['title_desc_sent_1', 'title_desc']]
# test_df.columns= ['label', 'text']
# test_df.tail()


# train_df.to_csv( new_data_folder + "train.csv",  sep='\t', index = False, header = True)
# dev_df.to_csv( new_data_folder + "dev.csv",sep='\t', index = False, header = True)
# test_df.to_csv( new_data_folder + "test.csv", sep='\t',index = False, header = True)

In [10]:
name_column_map = { 5: "label_topic", 3: "text"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                        name_column_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-05-28 05:11:14,611 Reading data from /content/drive/My Drive/Capstone/undersampled/interest_rates
2020-05-28 05:11:14,612 Train: /content/drive/My Drive/Capstone/undersampled/interest_rates/train.csv
2020-05-28 05:11:14,613 Dev: /content/drive/My Drive/Capstone/undersampled/interest_rates/dev.csv
2020-05-28 05:11:14,613 Test: /content/drive/My Drive/Capstone/undersampled/interest_rates/test.csv


#### Fine-tuning

In [11]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-28 05:11:19,312 loading file /content/drive/My Drive/Capstone/best-model.pt


In [12]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-28 05:11:28,444 ----------------------------------------------------------------------------------------------------
2020-05-28 05:11:28,448 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [2.2023022174835205,
  1.5165293216705322,
  2.4093642234802246,
  1.9244276285171509,
  1.6674854755401611,
  1.675127625465393,
  1.4697805643081665,
  1.5891019105911255,
  1.633575201034546,
  1.59573233127594],
 'dev_score_history': [0.5,
  0.7666666666666667,
  0.7,
  0.7,
  0.6333333333333333,
  0.7,
  0.5,
  0.7,
  0.7,
  0.7],
 'test_score': 0.7666666666666667,
 'train_loss_history': [1.116904228925705,
  0.9244091212749481,
  0.6702499091625214,
  1.0853126049041748,
  0.5736981183290482,
  0.3892064094543457,
  0.38179463148117065,
  0.29414570331573486,
  0.2746358886361122,
  0.2761368751525879]}

#### GDP folder after second stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.16.20%20PM.png?token=AAAAOMQOWGANF2A55GDHKDS62WIOS)

### Second Stage Employment (fine tune on hand annotated datasets)

In [0]:
new_data_folder = './drive/My Drive/capstone/data/second_stage_employment/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-05-25 22:38:14,715 Reading data from drive/My Drive/capstone/data/second_stage_employment
2020-05-25 22:38:14,716 Train: drive/My Drive/capstone/data/second_stage_employment/train.csv
2020-05-25 22:38:14,717 Dev: drive/My Drive/capstone/data/second_stage_employment/dev.csv
2020-05-25 22:38:14,717 Test: drive/My Drive/capstone/data/second_stage_employment/test.csv


In [0]:
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-25 22:38:44,461 loading file ./drive/My Drive/capstone/data/best-model.pt


In [0]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-25 22:39:13,405 ----------------------------------------------------------------------------------------------------
2020-05-25 22:39:13,410 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [1.7820369005203247,
  1.2702070474624634,
  1.01650071144104,
  0.7919940948486328,
  0.6037646532058716,
  0.5808563232421875,
  0.46471381187438965,
  0.6194255352020264,
  0.6725401282310486,
  0.49661093950271606],
 'dev_score_history': [0.6097560975609756,
  0.6260162601626016,
  0.7560975609756098,
  0.7723577235772358,
  0.8699186991869918,
  0.8699186991869918,
  0.9024390243902439,
  0.8211382113821138,
  0.8536585365853658,
  0.9024390243902439],
 'test_score': 0.9024390243902439,
 'train_loss_history': [1.6321105480194091,
  1.2179521560668944,
  0.7784557580947876,
  0.4638586640357971,
  0.483644163608551,
  0.3443927586078644,
  0.24840295314788818,
  0.31318709850311277,
  0.16766838282346724,
  0.08277207612991333]}

### Employment folder after second stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.24.58%20PM.png?token=AAAAOMRAGJI4MX5IUK52VM262WISU)

### Second Stage Stock Market (fine tune on hand annotated datasets)

In [0]:
new_data_folder = './drive/My Drive/capstone/data/second_stage_stock/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-05-25 22:51:52,474 Reading data from drive/My Drive/capstone/data/second_stage_stock
2020-05-25 22:51:52,475 Train: drive/My Drive/capstone/data/second_stage_stock/train.csv
2020-05-25 22:51:52,476 Dev: drive/My Drive/capstone/data/second_stage_stock/dev.csv
2020-05-25 22:51:52,477 Test: drive/My Drive/capstone/data/second_stage_stock/test.csv


In [0]:
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-25 22:52:09,316 loading file ./drive/My Drive/capstone/data/best-model.pt


In [0]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-25 22:52:40,932 ----------------------------------------------------------------------------------------------------
2020-05-25 22:52:40,938 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [1.1762595176696777,
  0.8527119159698486,
  1.6501855850219727,
  0.7665729522705078,
  0.6028701066970825,
  0.7755216360092163,
  0.7523883581161499,
  0.7765190601348877,
  0.47441190481185913,
  0.3788141906261444],
 'dev_score_history': [0.5614035087719298,
  0.7368421052631579,
  0.543859649122807,
  0.7368421052631579,
  0.7894736842105263,
  0.7894736842105263,
  0.7543859649122807,
  0.7368421052631579,
  0.8421052631578947,
  0.8596491228070176],
 'test_score': 0.8596491228070176,
 'train_loss_history': [1.649899458885193,
  0.8378177523612976,
  0.7674612760543823,
  0.7331142544746398,
  0.37666028141975405,
  0.4678428113460541,
  0.5148231387138367,
  0.2228946179151535,
  0.36473034918308256,
  0.14165073335170747]}

#### Stock folder after second stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.28.07%20PM.png?token=AAAAOMRZQTYPKV4SLOVBE2K62WIXG)